In [20]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.stats import iqr
from sklearn.model_selection import GridSearchCV

In [21]:
def get_silverman_bandwidth(data):
    std_dev = np.std(data)
    interquartile_range = iqr(data)
    n = len(data)
    return 1.06 * min(std_dev, interquartile_range / 1.34) * n**(-1/5)



def get_transformation_even_risk(buy_hold_return_list,bandwidth=None,kernel='gaussian',beta=None,risk_bond=0.1):
    #this is just for creating even distribution
    lower_risk_bond=risk_bond/2
    upper_risk_bond=1-risk_bond/2
    upper_value=np.quantile(buy_hold_return_list,upper_risk_bond)
    lower_value=np.quantile(buy_hold_return_list,lower_risk_bond)
    if bandwidth is None:
        silverman_bandwidth =get_silverman_bandwidth(buy_hold_return_list)
        log_bandwidths = np.linspace(np.log10(0.01 * silverman_bandwidth), np.log10(10 * silverman_bandwidth), 100)
        bandwidths = 10 ** log_bandwidths
        kde = KernelDensity(kernel=kernel)
        grid = GridSearchCV(kde, {'bandwidth': bandwidths})
        grid.fit(np.array(buy_hold_return_list).reshape(-1, 1))
        bandwidth=grid.best_params_["bandwidth"]
    kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(np.array(buy_hold_return_list)[:, np.newaxis])
    log_density = kde.score_samples(np.array(buy_hold_return_list)[:, np.newaxis])
    density = np.exp(log_density)
    density=density/np.sum(density)
    weights=[]
    for return_rate,single_density in zip(buy_hold_return_list,density):
        if return_rate>=lower_value and return_rate<=upper_value:
            weights.append(1/single_density)
        else:
            weights.append(1)
    weights=np.array(weights)
    
    return weights

In [22]:
return_rate=[0.01,-0.01,0.02,0.015,0.001,0.032]
get_transformation_even_risk(return_rate)

1.0
density [0.20039626 0.11545632 0.18727892 0.19976957 0.17446768 0.12263125]
0.20039625764620342
0.18727892119077696
0.19976956617663333
0.17446768176586813
weights [4.99011315 1.         5.33962922 5.00576749 5.73172057 1.        ]


[]

In [23]:
return_rate=[0.01,-0.01,0.02,0.015,0.001,0.032]
get_transformation_even_risk(return_rate)

1.0
density [0.20039626 0.11545632 0.18727892 0.19976957 0.17446768 0.12263125]
0.20039625764620342
0.18727892119077696
0.19976956617663333
0.17446768176586813
weights [4.99011315 1.         5.33962922 5.00576749 5.73172057 1.        ]


[]